<a href="https://colab.research.google.com/github/hijera/NN_colab_hw/blob/main/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BF%D0%BE_%D1%82%D0%B5%D0%BC%D0%B5_%C2%AB%D0%9A%D0%BE%D0%BB%D0%BB%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%B8%D0%B2%D0%BD%D0%B0%D1%8F_%D1%84%D0%B8%D0%BB%D1%8C%D1%82%D1%80%D0%B0%D1%86%D0%B8%D1%8F%C2%BB_(%D0%92%D0%BB%D0%B0%D1%81%D0%BE%D0%B2_%D0%90%D0%BD%D1%82%D0%BE%D0%BD).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Пакет SURPRISE:

1.  используйте данные MovieLens 1M,
2.  можно использовать любые модели из пакета,
3.  получите RMSE на тестовом сете 0,87 и ниже.

Комментарий преподавателя:
В домашнем задании на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе Cross-validation (5 фолдов), а не на отложенном датасете.


In [ ]:
!pip install surprise

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline

#Импорт surprise
from surprise import KNNWithMeans, KNNBasic,SVD,BaselineOnly
from surprise import NormalPredictor,KNNWithZScore,SVDpp,NMF,SlopeOne,CoClustering
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
%matplotlib inline

In [ ]:
#data = Dataset.load_builtin('ml-100k')
!wget  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip -O ml-latest.zip
#!wget https://files.grouplens.org/datasets/movielens/ml-latest.zip -O ml-latest.zip
!unzip -o ml-latest.zip

#!rm ./ml-latest -Rf
#!mv ./ml-latest-small ./ml-latest
prefix = 'ml-latest-small'
movies = pd.read_csv(os.path.join(prefix, 'movies.csv'))
ratings = pd.read_csv(os.path.join(prefix, 'ratings.csv'))


--2023-12-05 21:15:31--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: 'ml-latest.zip'

     0K .......... .......... .......... .......... ..........  5%  179K 5s
    50K .......... .......... .......... .......... .......... 10%  363K 4s
   100K .......... .......... .......... .......... .......... 15% 11,6M 2s
   150K .......... .......... .......... .......... .......... 20%  182K 3s
   200K .......... .......... .......... .......... .......... 26% 90,2M 2s
   250K .......... .......... .......... .......... .......... 31%  337M 2s
   300K .......... .......... .......... .......... .......... 36% 5,22M 1s
   350K .......... .......... .......... .......... .......... 41% 1,34M 1s
   400K .......... ......

Archive:  ml-latest.zip
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [ ]:
data_movielens=ratings.copy()
data_movielens=data_movielens.drop(columns=['timestamp'])
data_movielens

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [ ]:

#data_movielens=pd.DataFrame(data.__dict__['raw_ratings'], columns=['user_id','item_id','rating','timestamp'])
#data_movielens=data_movielens.drop(columns=['timestamp'])
data_movielens

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [ ]:
#jdata=ratings.join(movies.set_index('movieId'),on='movieId')
#jdata_scoped=jdata[['userId','title','rating']]
#jdata_scoped

In [ ]:
reader = Reader(rating_scale=(data_movielens.rating.min(), data_movielens.rating.max()))
data = Dataset.load_from_df(data_movielens, reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=.20,random_state=1)

In [ ]:
armse={}

algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)
# Run 5-fold cross-validation and print results
d=cross_validate(algo, data, measures=["RMSE", "MAE"], cv=5, verbose=True)
print(d)


RMSE: 0.8747
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8729  0.8710  0.8732  0.8730  0.8762  0.8733  0.0017  
MAE (testset)     0.6720  0.6679  0.6661  0.6716  0.6736  0.6702  0.0028  
Fit time          0.67    0.67    0.68    0.68    0.65    0.67    0.01    
Test time         0.08    0.08    0.23    0.09    0.08    0.11    0.06    
{'test_rmse': array([0.87294723, 0.87095616, 0.87324933, 0.87304089, 0.87616418]), 'test_mae': array([0.67201182, 0.66794038, 0.66606027, 0.67159369, 0.67359505]), 'fit_time': (0.6660370826721191, 0.6716041564941406, 0.6757509708404541, 0.679375171661377, 0.6509993076324463), 'test_time': (0.07599973678588867, 0.08100080490112305, 0.2259988784790039, 0.08524632453918457, 0.0840003490447998)}


In [ ]:
svd_rmse=np.mean(d['test_rmse'])
armse['SVD']=svd_rmse


RMSE вышло больше **0.87**, а нам нужно меньше. попробуем другие алгоритмы.

In [ ]:
another_algos=[KNNWithMeans(),
               KNNBasic(random_state=42),
               BaselineOnly(),
               NormalPredictor(),
               KNNWithZScore(random_state=42),
               SVDpp(random_state=42,n_epochs=4),NMF(),
               SlopeOne(),CoClustering(random_state=42)]
for algo in another_algos:
  print("Algo: "+algo.__class__.__name__)
  algo.fit(trainset)
  predictions = algo.test(testset)

  # Then compute RMSE
  accuracy.rmse(predictions)
  # Run 5-fold cross-validation and print results
  d=cross_validate(algo, data, measures=["RMSE", "MAE"], cv=5, verbose=True)
  armse[algo.__class__.__name__]=np.mean(d['test_rmse'])
  print(" ")

Algo: KNNWithMeans
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9001
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8939  0.8959  0.9030  0.8955  0.8978  0.8972  0.0032  
MAE (testset)     0.6809  0.6867  0.6895  0.6851  0.6852  0.6855  0.0028  
Fit time          0.10    0.12    0.13    0.13    0.12    0.12    0.01    
Test time         0.78    0.78    0.78    0.79    0.74    0.78    0.02    
 
Algo: KNNBasic
Computing the msd similarity matrix...
Done computing similarity matrix.
RM

Все варианты больше 0.87. Попробуем RandomizedSearchCV, и поперебираем гиперпараметры

In [ ]:
from surprise.model_selection import GridSearchCV,RandomizedSearchCV

In [ ]:

param_grid = {"n_epochs": [5,10,30], "reg_all": [0.001,0.01,0.1],"cache_ratings":[True],"lr_all":[0.0001,0.005,0.01,0.05],"n_factors":[10,20,40],"random_state":[1]} #,"cache_ratings":[True]
gs = RandomizedSearchCV(SVDpp, param_grid, measures=["rmse","mae","fcp","mse"], cv=5,n_jobs=-1,refit=False,joblib_verbose=6) #,"mae","mse","fcp" #,n_iter=1000
#(cache_ratings=True,random_state=42)

gs.fit(data)

# best RMSE score
print(gs.best_score["rmse"])
armse['RandomizedSearchCV_SVDPP']=gs.best_score["rmse"]
# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   48.4s
[Parallel(n_jobs=-1)]: Done  28 out of  50 | elapsed:  1.2min remaining:   54.4s
[Parallel(n_jobs=-1)]: Done  37 out of  50 | elapsed:  1.7min remaining:   35.8s
[Parallel(n_jobs=-1)]: Done  46 out of  50 | elapsed:  2.1min remaining:   11.1s


0.8614932300617199
{'n_epochs': 10, 'reg_all': 0.1, 'cache_ratings': True, 'lr_all': 0.05, 'n_factors': 40, 'random_state': 1}


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  2.2min finished


In [ ]:
print('| Метод             |  RMSE  |')
print('|-------------------|--------|')
print(f'| SVD               | '+str('{:.4f}'.format(armse['SVD']))+' |')
print(f'| KNNWithMeans      | '+str('{:.4f}'.format(armse['KNNWithMeans']))+' |')
print(f'| BaselineOnly      | '+str('{:.4f}'.format(armse['BaselineOnly']))+' |')
print(f'| KNNBasic          | '+str('{:.4f}'.format(armse['KNNBasic']))+' |')
print(f'| NormalPredictor   | '+str('{:.4f}'.format(armse['NormalPredictor']))+' |')
print(f'| KNNWithZScore     | '+str('{:.4f}'.format(armse['KNNWithZScore']))+' |')
print(f'| SVDpp             | '+str('{:.4f}'.format(armse['SVDpp']))+' |')
print(f'| SlopeOne          | '+str('{:.4f}'.format(armse['SlopeOne']))+' |')
print(f'| CoClustering      | '+str('{:.4f}'.format(armse['CoClustering']))+' |')
print(f'| SearchCV SVDpp    | '+str('{:.4f}'.format(armse['RandomizedSearchCV_SVDPP']))+' |')

| Метод             |  RMSE  |
|-------------------|--------|
| SVD               | 0.8733 |
| KNNWithMeans      | 0.8972 |
| BaselineOnly      | 0.8727 |
| KNNBasic          | 0.9482 |
| NormalPredictor   | 1.4260 |
| KNNWithZScore     | 0.8960 |
| SVDpp             | 0.8855 |
| SlopeOne          | 0.9021 |
| CoClustering      | 0.9463 |
| SearchCV SVDpp    | 0.8615 |
